In [1]:
import tensorflow as tf
from moviepy.editor import VideoFileClip
import numpy as np
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
# import cv2
import random
from tqdm import tqdm
from pathlib import Path
from glob import glob
from collections import defaultdict
import pickle as pkl


tqdm.pandas()

2023-02-03 07:15:44.402370: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import tensorflow_hub as hub
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# !wget https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
# import nltk
# from nltk.corpus import stopwords
import tensorflow.keras as keras
from tqdm import tqdm
import pickle
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
import itertools
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig


# def unicode_to_ascii(s):
#     return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# # def clean_stopwords_shortwords(w):
# #     stopwords_list=stopwords.words('english')
# #     words = w.split() 
# #     clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
# #     return " ".join(clean_words) 

# def preprocess_sentence(w):
#     w = unicode_to_ascii(w.lower().strip())
#     w = re.sub(r"([?.!,¿])", r" ", w)
#     w = re.sub(r'[" "]+', " ", w)
#     w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
#     # w=clean_stopwords_shortwords(w)
#     w=re.sub(r'@\w+', '',w)
#     return w

  warnings.warn(

  warnings.warn(



In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

2023-02-03 07:15:52.199281: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-03 07:15:52.312413: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-02-03 07:15:52.312494: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-03 07:15:52.312613: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-02-03 07:15:52.312642: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-02-03 07:15:52.314123: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

In [6]:
train_df = pd.read_csv("first_impression_text_train_annot.csv")

# read test data
test_df = pd.read_csv("first_impression_text_val_annot.csv")

In [7]:
train_df.head()

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,file_path_video,file_path_audio,transcript,filenames
0,0.523364,0.552083,0.626374,0.601942,0.504673,0.488889,/mnt/FirstImpression/unzippedData/training80_6...,/mnt/FirstImpression/audio_extract/training80_...,while he's cutting it and then turn around an...,J4GQm9j0JZ0.003
1,0.345794,0.375000,0.472527,0.582524,0.457944,0.366667,/mnt/FirstImpression/unzippedData/training80_7...,/mnt/FirstImpression/audio_extract/training80_...,responsibility to house the organ I had been ...,zEyRyTnIw5I.005
2,0.252336,0.291667,0.406593,0.485437,0.373832,0.511111,/mnt/FirstImpression/unzippedData/training80_5...,/mnt/FirstImpression/audio_extract/training80_...,I actually got quite a few sets of black pens...,nskJh7v6v1U.004
3,0.457944,0.489583,0.505495,0.398058,0.457944,0.377778,/mnt/FirstImpression/unzippedData/training80_3...,/mnt/FirstImpression/audio_extract/training80_...,I eat a lot and so I'd like a lot of foods. I...,6wHQsN5g2RM.000
4,0.607477,0.489583,0.406593,0.621359,0.570093,0.622222,/mnt/FirstImpression/unzippedData/training80_7...,/mnt/FirstImpression/audio_extract/training80_...,ask you guys to leave a question in the comme...,dQOeQYWIgm8.000


In [8]:
test_df.head()

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,file_path_video,file_path_audio,transcript,filenames
0,0.644860,0.593750,0.615385,0.640777,0.616822,0.555556,/mnt/FirstImpression/unzippedData/validation80...,/mnt/FirstImpression/audio_extract/validation8...,It's been a great journey so far. Hopefully w...,modNfUPt3F4.002
1,0.439252,0.520833,0.417582,0.572816,0.439252,0.411111,/mnt/FirstImpression/unzippedData/validation80...,/mnt/FirstImpression/audio_extract/validation8...,more pulling movements and then for chest day...,h6LOjpCRXtY.005
2,0.457944,0.312500,0.428571,0.398058,0.373832,0.555556,/mnt/FirstImpression/unzippedData/validation80...,/mnt/FirstImpression/audio_extract/validation8...,"Eww. But, um, yeah. Some people have been ask...",WER4ww680QQ.004
3,0.364486,0.572917,0.527473,0.553398,0.523364,0.322222,/mnt/FirstImpression/unzippedData/validation80...,/mnt/FirstImpression/audio_extract/validation8...,"I am a makeup artist, I am an esthetician, I ...",c4XnKouozXU.002
4,0.345794,0.468750,0.516484,0.417476,0.383178,0.477778,/mnt/FirstImpression/unzippedData/validation80...,/mnt/FirstImpression/audio_extract/validation8...,stops and Minnie agrees yeah it just sucks it...,OEKg-Tvwcbk.002


# load bert

In [9]:
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

num_classes = 5

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=num_classes)

loading file vocab.txt from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-

# generate features

In [10]:
def extract_embedding_text(df, output_fl = "bert_feature.pkl"):
    # fill na with empty string
    df.fillna("", inplace=True)
    
    feature = defaultdict(dict)
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    for i in tqdm(df.itertuples(), total=len(df)):
        # get transcript
        sent = i.transcript
        
        # get input id and mask for bert model
        bert_inp = bert_tokenizer.encode_plus(sent,add_special_tokens = True,
                                              max_length =64,pad_to_max_length = True,
                                              return_attention_mask = True)    
        # feature dict
        feature[i.filenames] = {"input_ids": bert_inp['input_ids'], "attention_mask": bert_inp['attention_mask']}

    with open(output_fl, 'wb') as f:
        pkl.dump(dict(feature), f)


In [11]:
extract_embedding_text(train_df)

loading file vocab.txt from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-

In [12]:
# dict(feature)

In [141]:
def get_feature_tf_data(filenames, labels):
    with open("bert_feature.pkl", 'rb') as f:
        train_feature = pkl.load(f)
    filenames = filenames.numpy().decode('utf-8')
    return np.array(train_feature[filenames]['input_ids']), np.array(train_feature[filenames]['attention_mask']) ,labels

In [142]:
# def get_feature_tf_map(filenames, labels):
#     return tf.py_function(func=get_feature_tf, inp=[filenames, labels], Tout=[tf.int64, tf.int64, tf.float64])


In [143]:
# get test features
extract_embedding_text(test_df, output_fl = "bert_feature_test.pkl")

loading file vocab.txt from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-

In [144]:
# load train and test feature
import pickle as pkl


with open("bert_feature.pkl", 'rb') as f:
    feature = pkl.load(f)

    
with open("bert_feature_test.pkl", 'rb') as f:
    test_feature = pkl.load(f)


In [145]:
# create tf dataset feature for test data
def get_feature_tf_data_test(filenames, labels):
    with open("bert_feature_test.pkl", 'rb') as f:
        train_feature = pkl.load(f)
    filenames = filenames.numpy().decode('utf-8')
    return np.array(train_feature[filenames]['input_ids']), np.array(train_feature[filenames]['attention_mask']), labels 

# Create tf dataset

In [146]:
ft = train_df['filenames'].values
labels = train_df[['openness',  'conscientiousness', 'extraversion', 'agreeableness','neuroticism']]

ft_test = test_df['filenames'].values
labels_test = test_df[['openness',  'conscientiousness', 'extraversion', 'agreeableness','neuroticism']]

In [147]:
len(ft)

6000

In [148]:
np.array(feature['J4GQm9j0JZ0.003']['attention_mask']).dtype

dtype('int64')

In [149]:
def create_multiinp(ft, label):
    print(ft.numpy())
    return ft, label



In [150]:
# split data into train and test
from sklearn.model_selection import train_test_split


train_x, test_x, train_y, test_y = train_test_split(ft, labels, test_size=0.1)

In [151]:
# define batch size and steps per epochs
train_batch_size = val_batch_size = 8


train_steps = len(train_x)//train_batch_size
val_steps = len(test_x)//val_batch_size

AUTOTUNE = tf.data.AUTOTUNE

In [164]:
# create tf dataset for training
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))

# added tf.py_function to get string from tensor as well to get feature from dict
train_ds = train_ds.map(lambda x,y: tf.py_function(get_feature_tf_data, [x, y], [tf.int64, tf.int64, tf.float64]), AUTOTUNE)

# for multi input create dict having input name
train_ds = train_ds.map(lambda x,y,z : ({"input_token": x, "masked_token": y}, z))


# create tf dataset for validation
val_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
val_ds = val_ds.map(lambda x,y: tf.py_function(get_feature_tf_data, [x, y], [tf.int64, tf.int64, tf.float64]), AUTOTUNE)
# for multi input create dict having input name
val_ds = val_ds.map(lambda x,y,z : ({"input_token": x, "masked_token": y}, z))


# create tf dataset for validation
test_ds = tf.data.Dataset.from_tensor_slices((ft_test, labels_test))
test_ds = test_ds.map(lambda x,y: tf.py_function(get_feature_tf_data_test, [x, y], [tf.int64, tf.int64, tf.float64]), AUTOTUNE)
# for multi input create dict having input name
test_ds = test_ds.map(lambda x,y,z : ({"input_token": x, "masked_token": y}, z))


In [165]:
for i in train_ds:
    print(i)
    break

({'input_token': <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([  101,  1045,  2812,  2045,  1005,  1055,  1037,  2843,  1997,
        2428,  4658,  4933,  2006,  2026, 10474,  1010,  2061,  2057,
        1005,  2128,  6069,  2203,  2009,  2045,  1012,  3246,  2017,
        4364,  5632,  1012,  1045,  1005,  2222,  2156,  2017,  2101,
        1012,  9765, 22591,  2015,  2203, 10976,   999,   102,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])>, 'masked_token': <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>}, <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.52222222, 0.45631068, 0.48598131, 0.42857143, 0.45833333])>)


In [166]:
for i in val_ds:
    print()
    break

In [167]:
train_ds = train_ds.shuffle(32).repeat().batch(train_batch_size).prefetch(AUTOTUNE)
val_ds = val_ds.batch(val_batch_size).prefetch(AUTOTUNE)
test_ds = test_ds.batch(val_batch_size).prefetch(AUTOTUNE)

# Generate model architechture

In [168]:
import tensorflow.keras.backend as k
def mean_acc(y_true, y_pred):
    diff = k.abs(y_true - y_pred)
    return k.mean(1-diff)

In [169]:
import os

def generate_output_dir(outdir, run_desc):
    prev_run_dirs = []
    if os.path.isdir(outdir):
        prev_run_dirs = [x for x in os.listdir(outdir) if os.path.isdir(\
            os.path.join(outdir, x))]
    prev_run_ids = [re.match(r'^\d+', x) for x in prev_run_dirs]
    prev_run_ids = [int(x.group()) for x in prev_run_ids if x is not None]
    cur_run_id = max(prev_run_ids, default=-1) + 1
    run_dir = os.path.join(outdir, f'{cur_run_id:05d}-{run_desc}')
    assert not os.path.exists(run_dir)
    os.makedirs(run_dir)
    return run_dir

In [170]:
outdir = "./checkpoint/"
run_desc = "text_personality"

run_dir = generate_output_dir(outdir, run_desc)
print(f"Results saved to: {run_dir}")

Results saved to: ./checkpoint/00010-text_personality


In [171]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

class MyModelCheckpoint(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch,logs)

        # Also save the optimizer state
        filepath = self._get_file_path(epoch=epoch, 
                                       logs=logs, 
                                       # batch=None
                                      )

        filepath = filepath.rsplit( ".", 1 )[ 0 ] 
        filepath += ".pkl"

        with open(filepath, 'wb') as fp:
            pickle.dump(
            {
                'opt': model.optimizer.get_config(),
                'epoch': epoch+1,
                'lr': model.optimizer.learning_rate
                
             # Add additional keys if you need to store more values
            }, fp, protocol=pickle.HIGHEST_PROTOCOL)
        print('\nEpoch %05d: saving optimizaer to %s' % (epoch + 1, filepath))

In [172]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


checkpoint = MyModelCheckpoint(os.path.join(run_dir, 'text-personality-model-{epoch:02d}-{val_loss:.2f}.h5'),
        monitor='val_loss',verbose=1, save_best_only=True, mode='auto')


# reduce lr on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_mean_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='min', min_lr=0.0000000001)

callbacks_list = [checkpoint, reduce_lr]

In [173]:
import transformers

transformer_model = transformers.TFBertModel.from_pretrained('bert-base-uncased')

transformer_model.trainable = True

input_ids = tf.keras.layers.Input(shape=(64), name='input_token', dtype=tf.int32)

input_masks_ids = tf.keras.layers.Input(shape=(64), name='masked_token', dtype=tf.int32)

encodings = transformer_model(input_ids, input_masks_ids)[0]

# drop1 = tf.keras.layers.Dropout(0.2)(encodings)

avg_pooling = tf.keras.layers.GlobalAvgPool1D()(encodings)


output_ly = tf.keras.layers.Dense(5, activation='sigmoid')(avg_pooling)


model = tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = output_ly)

loading configuration file config.json from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file tf_model.h5 from cache at /home/ayush/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tf_model.h5
Some layers from the model chec

In [174]:
# model.predict([val_inp,val_mask])

In [175]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   TFBaseModelOutputWit 109482240   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_3 (Glo (None, 768)          0           tf_bert_model_3[0][0]      

In [176]:
model.get_layer("input_token")

In [177]:
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bert_model.h5'

print('\nBert Model',model.summary())

# loss = [tf.keras.losses.BinaryCrossentropy(from_logits=False)]
loss = ['mae']

metric = [mean_acc]
optimizer = tf.keras.optimizers.Adam()

model.compile(loss=loss,optimizer=optimizer,metrics=metric)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   TFBaseModelOutputWit 109482240   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_3 (Glo (None, 768)          0           tf_bert_model_3[0][0]      

In [ ]:
model.fit(x= train_ds, steps_per_epoch=train_steps, 
               validation_data=val_ds,validation_steps=val_steps,
               epochs=50, callbacks=callbacks_list)

Epoch 1/50


173/675 [======>.......................] - ETA: 4:40 - loss: 0.1727 - mean_acc: 0.8273

In [ ]:
# history=model.fit([train_inp,train_mask],
#                        train_label,batch_size=8,
#                        epochs=50,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks_list)

In [68]:
# mount1/Harish/Personality_trait_Exp/checkpoint/00000-text_personality/text-personality-model-04-0.12.h5

In [69]:
# # load trained model state
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# import pickle

# def load_model_data(model_path, opt_path, custom_objects):
#     model = load_model(model_path, custom_objects)
#     with open(opt_path, 'rb') as fp:
#         d = pickle.load(fp)
#         epoch = d['epoch']
#         opt = d['opt']
#     return epoch, model, opt

# epoch, model_env, opt = load_model_data('/mount1/mount1/Harish/Personality_trait_Exp/checkpoint/00000-text_personality/text-personality-model-04-0.12.h5',
#                                         '/mount1/mount1/Harish/Personality_trait_Exp/checkpoint/00000-text_personality/text-personality-model-04-0.12.pkl', custom_objects={"mean_acc": mean_acc, "TFBertModel": transformers.TFBertModel})

# model_env.compile(
#     loss = ['mae'],
#     # loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
#     optimizer = tf.keras.optimizers.Adam.from_config(opt), 
#     metrics = [mean_acc]
# )

# model_env.fit([train_inp,train_mask],
#               train_label,batch_size=8,
#               epochs=50, validation_data=([val_inp,val_mask],val_label),initial_epoch=epoch, callbacks=callbacks_list)

# Saving best model

In [70]:
# mount1/Harish/Personality_trait_Exp/checkpoint/00001-text_personality/text-personality-model-10-0.12.h5

In [76]:
# import transformers
# import tensorflow.keras.backend as k
# from tensorflow.keras.models import load_model


# def mean_acc(y_true, y_pred):
#     diff = k.abs(y_true - y_pred)
#     return k.mean(1-diff)


# model_path = "/mount1/mount1/Harish/Personality_trait_Exp/checkpoint/00003-text_personality/text-personality-model-08-0.12.h5"

# custom_objects={"mean_acc": mean_acc, "TFBertModel": transformers.TFBertModel}
# model = load_model(model_path, custom_objects)

Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [77]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_4 (TFBertModel)   TFBaseModelOutputWit 109482240   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 768)          0           tf_bert_model_4[0][0]      

In [78]:
# model.save("text_personality_ocean.h5")

In [79]:
# model.save("text_personality_ocean")

2023-02-01 15:38:20.258704: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: text_personality_ocean/assets


INFO:tensorflow:Assets written to: text_personality_ocean/assets
